In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow import keras

## Data Hotel Goibibo 

In [2]:
df_goibibo = pd.read_csv('goibibo_com-travel_sample.csv')
# select potential parameter only
df_goibibo.drop(columns=['room_type','room_count','additional_info','province','area','locality','crawl_date','hotel_brand','hotel_category','pageurl','qts','query_time_stamp','room_area','similar_hotel','sitename','uniq_id','image_count'], inplace=True)
# drop data review
df_goibibo.drop(columns=['review_count_by_category','site_review_count','site_stay_review_rating','guest_recommendation'], inplace=True)

df_goibibo.head(3)

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_facilities,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,Doctor on Call|Dry Cleaning|Laundry Service Av...,2,32.139387,77.154660,Hadimba Temple|Naggar Village|Himalayan Nyingm...,HTLGBO1000016238,Baragarh Regency,Resort,Room Service |Basic Bathroom Amenities|Cable /...,4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,Airport Transfer Available / Surcharge|Banquet...,0,28.472097,77.072546,Sushant Lok|Sahara Mall|Amity International Sc...,HTLGBO1000015822,Asian Suites A- 585,Guest House,Room Service |Air Conditioning |Basic Bathroom...,4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,Swimming Pool|Bar / Lounge |Laundry Service Av...,0,15.548398,73.757634,"Anjuna Beach|Calangute Beach|Titos lane, baga|...",HTLGBO1000004551,Bevvan Resort,Resort,Room Service |Air Conditioning |Cable / Satell...,2.5,Goa


In [3]:
# eksplorasi dataset dan drop null serta duplikat
print('jumlah property id: ',len(df_goibibo['property_id'].unique()))
print('jumlah property name: ',len(df_goibibo['property_name'].unique()))
print('jumlah property type: ',len(df_goibibo['property_type'].unique()))

df_goibibo.dropna(inplace=True)
df_goibibo.drop_duplicates(subset=['property_name'], inplace=True)
print('jumlah data setelah drop null dan duplicates: ',len(df_goibibo.dropna()))

jumlah property id:  4000
jumlah property name:  3805
jumlah property type:  18
jumlah data setelah drop null dan duplicates:  1953


In [4]:
# eksplorasi lanjutan
print('Daftar tipe properti: \n', len(df_goibibo['property_type'].unique()), df_goibibo['property_type'].unique())
print('Total kota: ',len(df_goibibo['city'].unique()))
print('Total provinsi: ',len(df_goibibo['state'].unique()), df_goibibo['state'].unique())
print('Daftar hotel rating: ',len(df_goibibo['hotel_star_rating'].unique()), df_goibibo['hotel_star_rating'].unique())

# room type terlalu beragam, tidak standar jenisnya
# print('Daftar Room Type: ',len(df_goibibo['room_type'].unique()))

Daftar tipe properti: 
 17 ['Resort' 'Guest House' 'Cottage' 'Hotel' 'Homestay' 'Palace'
 'Service Apartment' 'Hostel' 'Lodge' 'Tent' 'Villa' 'Houseboat'
 'Bungalow' 'BnB' 'Beach Hut' 'Motel' 'Farm Stay']
Total kota:  330
Total provinsi:  27 ['Himachal Pradesh' 'Haryana' 'Goa' 'Delhi' 'Rajasthan' 'Assam'
 'Tamil Nadu' 'Gujarat' 'West Bengal' 'Maharashtra' 'Karnataka' 'Kerala'
 'Punjab' 'Uttarakhand' 'Madhya Pradesh' 'Uttar Pradesh' 'Sikkim'
 'Andaman and Nicobar Islands' 'Andhra Pradesh' 'Orissa'
 'Jammu and Kashmir' 'Jharkhand' 'Pondicherry' 'Bihar' 'Meghalaya'
 'Dadra and Nagar Haveli' 'Chhattisgarh']
Daftar hotel rating:  6 [2 0 3 1 4 5]


In [5]:
# melihat tipe data
df_goibibo.dtypes

address                object
city                   object
country                object
hotel_description      object
hotel_facilities       object
hotel_star_rating       int64
latitude              float64
longitude             float64
point_of_interest      object
property_id            object
property_name          object
property_type          object
room_facilities        object
site_review_rating    float64
state                  object
dtype: object

In [6]:
# change nested data to array
df_goibibo['hotel_facilities'] = [item.split("|") for item in df_goibibo['hotel_facilities']]
df_goibibo['point_of_interest'] = [item.split("|") for item in df_goibibo['point_of_interest']]
df_goibibo['room_facilities'] = [item.split("|") for item in df_goibibo['room_facilities']]
df_goibibo

,address,city,country,hotel_description,hotel_facilities,hotel_star_rating,latitude,longitude,point_of_interest,property_id,property_name,property_type,room_facilities,site_review_rating,state
0,"15th Mile, N.H.21,Manali, District Kullu,Himac...",Manali,India,The standard check-in time is 12:00 PM and the...,"[Doctor on Call, Dry Cleaning, Laundry Service...",2,32.139387,77.154660,"[Hadimba Temple, Naggar Village, Himalayan Nyi...",HTLGBO1000016238,Baragarh Regency,Resort,"[Room Service , Basic Bathroom Amenities, Cabl...",4.0,Himachal Pradesh
1,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...",Gurgaon,India,The standard check-in time is 12:00 PM and the...,"[Airport Transfer Available / Surcharge, Banqu...",0,28.472097,77.072546,"[Sushant Lok, Sahara Mall, Amity International...",HTLGBO1000015822,Asian Suites A- 585,Guest House,"[Room Service , Air Conditioning , Basic Bathr...",4.5,Haryana
2,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...",Goa,India,The standard check-in time is 12:00 PM and the...,"[Swimming Pool, Bar / Lounge , Laundry Service...",0,15.548398,73.757634,"[Anjuna Beach, Calangute Beach, Titos lane, ba...",HTLGBO1000004551,Bevvan Resort,Resort,"[Room Service , Air Conditioning , Cable / Sat...",2.5,Goa
3,Simsa,Manali,India,The standard check-in time is 12:00 PM and the...,"[Doctor on Call, Dry Cleaning, Laundry Service...",2,32.223603,77.185900,"[The Mall, Tibetian Monastery, Vashisht Hot Ba...",HTLGBO1000007993,Apple Inn Cottage,Cottage,"[Basic Bathroom Amenities, Cable / Satellite /...",5.0,Himachal Pradesh
4,"8180 Street No.-6,Arakashan Road,Paharganj",Delhi,India,The standard check-in time is 12:00 PM and the...,"[Internet Access - Surcharge, Laundry Service ...",2,28.646777,77.212735,"[Gaffar Market, YMCA Institute, Agrasen Boali,...",HTLGBO1000008281,Anmol Hotel Pvt.Ltd,Hotel,"[Basic Bathroom Amenities, Cable / Satellite /...",2.8,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,"sarvaritu vilas , inside udaipole , opp sagasj...",Udaipur,India,The standard check-in time is 12:00 PM and the...,"[Airport Transfer Available / Surcharge, Docto...",2,24.575598,73.696959,"[Jagdish Temple, City Palace, Udaipur Bus Stan...",HTLGBO1000012898,Hotel Shubh Laxmi,Hotel,"[Room Service , Basic Bathroom Amenities, Cabl...",4.2,Rajasthan
3993,"Near Yasholaxmi Nagar, Kolhapur Rd,",Ichalkaranji,India,The standard check-in time is 10:00 AM and the...,"[Bar / Lounge , Doctor on Call, Dry Cleaning, ...",2,16.690042,74.456394,[Yasholaxmi Nagar],HTLGBO1000083318,Hotel Signature,Hotel,"[Room Service , Air Conditioning , Alarm Clock...",5.0,Maharashtra
3994,"Near Railway Station , Kota Jn",Kota,India,The standard check-in time is 12:00 PM and the...,"[Bar / Lounge , Doctor on Call, Laundry Servic...",2,25.222694,75.878170,"[Kota Airport, Station Road Area]",HTLGBO1000009343,Hotel Shree Anand Restaurant &amp; Bar,Hotel,"[Room Service , Air Conditioning , Basic Bathr...",3.4,Rajasthan
3997,"Marol Maroshi Road, Near To Marol naka FireBri...",Mumbai,India,The standard check-in time is 12:00 PM and the...,"[Banquet Facilities, Bar / Lounge , Currency E...",3,19.110926,72.879038,"[Mount Mary Church, Juhu Beach, Domestic Airpo...",HTLGBO1000001584,Hotel Silver Inn,Hotel,"[Air Conditioning , Cable / Satellite / Pay TV...",4.0,Maharashtra


In [7]:
df_goibibo[['hotel_facilities','room_facilities','point_of_interest']]

,hotel_facilities,room_facilities,point_of_interest
0,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Room Service , Basic Bathroom Amenities, Cabl...","[Hadimba Temple, Naggar Village, Himalayan Nyi..."
1,"[Airport Transfer Available / Surcharge, Banqu...","[Room Service , Air Conditioning , Basic Bathr...","[Sushant Lok, Sahara Mall, Amity International..."
2,"[Swimming Pool, Bar / Lounge , Laundry Service...","[Room Service , Air Conditioning , Cable / Sat...","[Anjuna Beach, Calangute Beach, Titos lane, ba..."
3,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Basic Bathroom Amenities, Cable / Satellite /...","[The Mall, Tibetian Monastery, Vashisht Hot Ba..."
4,"[Internet Access - Surcharge, Laundry Service ...","[Basic Bathroom Amenities, Cable / Satellite /...","[Gaffar Market, YMCA Institute, Agrasen Boali,..."
...,...,...,...
3992,"[Airport Transfer Available / Surcharge, Docto...","[Room Service , Basic Bathroom Amenities, Cabl...","[Jagdish Temple, City Palace, Udaipur Bus Stan..."
3993,"[Bar / Lounge , Doctor on Call, Dry Cleaning, ...","[Room Service , Air Conditioning , Alarm Clock...",[Yasholaxmi Nagar]
3994,"[Bar / Lounge , Doctor on Call, Laundry Servic...","[Room Service , Air Conditioning , Basic Bathr...","[Kota Airport, Station Road Area]"
3997,"[Banquet Facilities, Bar / Lounge , Currency E...","[Air Conditioning , Cable / Satellite / Pay TV...","[Mount Mary Church, Juhu Beach, Domestic Airpo..."


In [8]:
# rearrange the feature position
df_goibibo = df_goibibo[['property_id','property_name','hotel_description','address','point_of_interest','city','state','country','latitude','longitude','property_type','hotel_star_rating','hotel_facilities','room_facilities','site_review_rating']]

# recreate the property_id for better indexing
hid = []
for n in range(1, len(df_goibibo)+1):
    hid.append('H{0:06}'.format(n))
df_goibibo['property_id'] = hid
# save data hotel for non ML
# df_goibibo.to_csv('DataHotel_nonML.csv')

In [9]:
df_goibibo.head(3)

,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,hotel_facilities,room_facilities,site_review_rating
0,H000001,Baragarh Regency,The standard check-in time is 12:00 PM and the...,"15th Mile, N.H.21,Manali, District Kullu,Himac...","[Hadimba Temple, Naggar Village, Himalayan Nyi...",Manali,Himachal Pradesh,India,32.139387,77.154660,Resort,2,"[Doctor on Call, Dry Cleaning, Laundry Service...","[Room Service , Basic Bathroom Amenities, Cabl...",4.0
1,H000002,Asian Suites A- 585,The standard check-in time is 12:00 PM and the...,"A-585, Sushant Lok-1 ,Near Iffco Chowk Metro S...","[Sushant Lok, Sahara Mall, Amity International...",Gurgaon,Haryana,India,28.472097,77.072546,Guest House,0,"[Airport Transfer Available / Surcharge, Banqu...","[Room Service , Air Conditioning , Basic Bathr...",4.5
2,H000003,Bevvan Resort,The standard check-in time is 12:00 PM and the...,"Cobra Vaddo,Calungate Baga Road, Bardez, Calan...","[Anjuna Beach, Calangute Beach, Titos lane, ba...",Goa,Goa,India,15.548398,73.757634,Resort,0,"[Swimming Pool, Bar / Lounge , Laundry Service...","[Room Service , Air Conditioning , Cable / Sat...",2.5


In [10]:
hotel = set()
for item in df_goibibo['hotel_facilities']:
    for h in item:
        hotel.add(h)
room = set()
for item in df_goibibo['room_facilities']:
    for r in item:
        room.add(r)
poi = set()
for item in df_goibibo['point_of_interest']:
    for p in item:
        poi.add(p)

In [11]:
print('Jumlah unique hotel facilities: ', len(hotel))
print('Jumlah unique room facilities: ', len(room))
print('Jumlah unique point of interest: ', len(poi))
# POI Terlalu beragam dan tidak standar sehingga tidak diperhitungkan dahulu dalam ML

Jumlah unique hotel facilities:  202
Jumlah unique room facilities:  166
Jumlah unique point of interest:  4218


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
# one-hot encoding hotel facilities
mlb = MultiLabelBinarizer()
x = mlb.fit_transform(df_goibibo['hotel_facilities'])
# one-hot encoding room facilities
mlb2 = MultiLabelBinarizer()
y = mlb2.fit_transform(df_goibibo['room_facilities'])

In [13]:
df_hotel_facilities = pd.DataFrame(x, columns=mlb.classes_)
df_room_facilities = pd.DataFrame(y, columns=mlb2.classes_)

In [14]:
df_hotel_facilities.head(3)

,Bike on Rent,Cook Services,Drivers Rest Room,24 Hour Front Desk,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Airport Transfer Available / Surcharge,Airport Transfer Free,Amphitheatre,Anti-Slip Ramps,Audio - Visual Equipment,Ayurvedic Facilities,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Boating,Bowling,Business Center,Business Center,CCTV,Camel Ride,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Conference Facility,Courier Service,Courtesy Coach,Currency Exchange,Dance Performances (on demand),Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Exit Signs,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Food Facility,Free Internet Access,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,Horse Ride ( Chargeable ),Hotel Trading License,In House Events,Internet / Fax (Reception area only),Internet Access,Internet Access,Internet Access - Surcharge,Internet Access Free,Jacuzzi,Karaoke,Karaoke,Kids Play Zone,Kitchen available (home cook food on request),Laptop Available on Hire,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Luggage Storage,Major Credit Cards Accepted,Marina On Site,Massage Services,Massage Services,Medical Assistance Available,Meditation Room,Metal Detectors,Mini Theatre,Multi - Line Phone,Multi Cuisine Restaurant,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Newspapers In Lobby,Night Club,Non Smoking Hotel,Non Smoking Rooms,Open Air Restaurant / Dining,Open Air Restaurant / Dining,Outdoor Parking - Non Secured,Outdoor Parking - Secured,Oxygen Cylinder (on request),Paid Transfers,Parking (Surcharge),Parking Facilities Available,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Cafe,Pool Table,Poolside Bar,Porter / Bellhop,Porter Services Available,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Room Service,Room Service,Room Service (24 Hours),STD / ISD,Safari,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Shopping Drop Facility (on fixed timings),Ski Storage,Skiing Facility Available,Smoke Detector,Smoking Area,Smoking Area,Smoking Rooms,Solarium,Spa On Site - Chargeable,Spa Services Nearby,Spa Services Onsite - Free,Speciality Restaurant,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suitable For Children,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Transfers Available,Translation Services,Trekking,Valet service,Veg / Non Veg Kitchens Separate,Vegetarian Food / Jain Food Available,Veranda,Village Visit ( Chargeable ),Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,Yoga Hall / Classes
0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [24]:
print(len(df_hotel_facilities.columns.values))
list(df_hotel_facilities.columns.values)

202


[' Bike on Rent',
 ' Cook Services',
 ' Drivers Rest Room',
 '24 Hour Front Desk',
 'ATM / Banking',
 'Access Via Exterior Corridors',
 'Adjoining Rooms',
 'Air / Rail Booking',
 'Air-Conditioning - Central',
 'Airport Transfer Available / Surcharge',
 'Airport Transfer Free',
 'Amphitheatre',
 'Anti-Slip Ramps',
 'Audio - Visual Equipment',
 'Ayurvedic Facilities',
 'Ayurvedic Facilities ',
 'Babysitting/ Child Care (surcharge)',
 'Badminttion Court',
 'Baggage Room',
 'Ballroom',
 'Banquet Facilities',
 'Bar / Lounge ',
 'Barbecue Area',
 'Basketball Court ',
 'Beauty Salon - on charge',
 'Billiards',
 'Board Room',
 'Boating',
 'Bowling',
 'Business Center',
 'Business Center ',
 'CCTV',
 'Camel Ride',
 'Campfire / Bon Fire',
 'Car Rental',
 'Casino',
 'Catering',
 'Cell Phone Rental',
 "Children's Park",
 'City Shuttle Service',
 'Clubhouse',
 'Coffee Shop / Cafe',
 'Computer Rental',
 'Concierge',
 'Conference Facility',
 'Conference Facility ',
 'Courier Service',
 'Courtesy Coac

In [15]:
df_room_facilities.head(3)

,Hot / Cold Running Water,120 AC,220 AC,Air Conditioning,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Balcony / Terrace,Basic Bathroom Amenities,Bath Tub,Bathrobe,Bathrobe (on request),Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast Available (surcharge),Breakfast chargeable,Cable / Satellite / Pay TV available,Cable/Satellite TV,Ceiling Fan,Centrally Heated,Channel Music,Citrus Juicer,Climate Control,Clock Radio,Closet,Clothes Dryer,Clothes Hook in Bathroom,Clothes Press,Coffee Table,Color TV,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Custom Duvets / Linen,Daily Room Cleaning,Desk,Desk in Room,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dry Cleaning,Dustbins,Electronic Weighing Machine,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Free Newspaper,Fruit Basket,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,HD / LCD Satellite TV,Hairdryer,Hairdryer (on request),Handheld Showers,Handicapped Facilities,Heating Facility (on request),Heating Facility (on request),Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-Room Safe,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Internet Access,Internet Connection In Room,Iron / Ironing board available,Iron/Ironing Board,Kitchenette,Laundry Service Available,Local Calls Free,Locker Facility,Luggage Rack,Luxurious Feather Mattresses,Makeup / Shaving Mirror,Microwave Oven Available,Mineral Water,Mini Bar,Mini Refrigerator,Mini bar - On Charge,Mirror,Modern Toilet Facilities,Moisturiser,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Pay Movies,Phone Messaging Services,Pillow menu,Pinup Boards,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Refrigerator,Remote Control TV,Remote Controlled Lights and Fan,Rollaway Beds,Room Door Fitted with Lock and Key,Room Heater,Room Service,Room Service,Room with Window/Ventilation,Rooms with Attached Bathrooms,Safe (on request),Safe - In - Room,Safe - In - Room,Sandwich Maker,Sanitary Bin,Second Bathrooms,Separate Bathtubs And Shower,Shared Bathroom,Shaving Kit /Dental Kit,Shoe Shine,Shower,Shower Cap,Shower Caps Available,Shower Facility Available,Slippers,Smoke Detector,Snacks / Beverage,Sofa Cum Bed Available,Sofa Table / Extra Seats,Sofa Table / Extra Seats,Soft Bar,Soundproof Windows,Split A.C's,Stationery Kit,Stove / Oven,Studio Apartment - Large,Sufficient Lighting,TV,Tea Coffee Kettle / Maker,Telephone,Telephone in Toilet,Television,Toaster,Toiletries,Two Line Telephone,Utensils (on request),Video/ DVD/ CD Player Available,Voicemail,Wardrobe,Washing machine,Western Toilet Seat,Window Opens,Work Station,Working Desk Phone
0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [23]:
print(len(df_room_facilities.columns.values))
list(df_room_facilities.columns.values)

166


[' Hot / Cold Running Water',
 '120 AC',
 '220 AC',
 'Air Conditioning',
 'Air Conditioning ',
 'Air Cooler',
 'Alarm Clock',
 'Attached Bathroom',
 'Balcony',
 'Balcony / Terrace',
 'Basic Bathroom Amenities',
 'Bath Tub',
 'Bathrobe',
 'Bathrobe (on request) ',
 'Bathrobes',
 'Bathroom Amenities',
 'Bathtub',
 'Bathtub Only',
 'Bedside Table',
 'Bidet',
 'Blackout Drapes',
 'Breakfast Available',
 'Breakfast Available (surcharge)',
 'Breakfast chargeable',
 'Cable / Satellite / Pay TV available',
 'Cable/Satellite TV',
 'Ceiling Fan',
 'Centrally Heated',
 'Channel Music',
 'Citrus Juicer',
 'Climate Control',
 'Clock Radio',
 'Closet',
 'Clothes Dryer',
 'Clothes Hook in Bathroom',
 'Clothes Press',
 'Coffee Table',
 'Color TV',
 'Complete Bedding',
 'Complimentary Use of Bicycle',
 'Computer',
 'Cribs Available',
 'Crockery / Cutlery',
 'Curtains/Shades',
 'Custom Duvets / Linen',
 'Daily Room Cleaning',
 'Desk',
 'Desk in Room',
 'Direct Dial Phone',
 'Disabled Friendly Bathrooms'

In [16]:
# merge dataframe utama dengan hotel facilities
df_final = pd.concat([df_goibibo.reset_index(drop=True),df_hotel_facilities.reset_index(drop=True)], axis=1)
print(df_final.shape)
df_final.head(3)

(1953, 217)


,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,hotel_facilities,room_facilities,site_review_rating,Bike on Rent,Cook Services,Drivers Rest Room,24 Hour Front Desk,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Airport Transfer Available / Surcharge,Airport Transfer Free,Amphitheatre,Anti-Slip Ramps,Audio - Visual Equipment,Ayurvedic Facilities,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Boating,Bowling,Business Center,Business Center,CCTV,Camel Ride,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Conference Facility,Courier Service,Courtesy Coach,Currency Exchange,Dance Performances (on demand),Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Exit Signs,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Food Facility,Free Internet Access,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,Horse Ride ( Chargeable ),Hotel Trading License,In House Events,Internet / Fax (Reception area only),Internet Access,Internet Access,Internet Access - Surcharge,Internet Access Free,Jacuzzi,Karaoke,Karaoke,Kids Play Zone,Kitchen available (home cook food on request),Laptop Available on Hire,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Luggage Storage,Major Credit Cards Accepted,Marina On Site,Massage Services,Massage Services,Medical Assistance Available,Meditation Room,Metal Detectors,Mini Theatre,Multi - Line Phone,Multi Cuisine Restaurant,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Newspapers In Lobby,Night Club,Non Smoking Hotel,Non Smoking Rooms,Open Air Restaurant / Dining,Open Air Restaurant / Dining,Outdoor Parking - Non Secured,Outdoor Parking - Secured,Oxygen Cylinder (on request),Paid Transfers,Parking (Surcharge),Parking Facilities Available,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Cafe,Pool Table,Poolside Bar,Porter / Bellhop,Porter Services Available,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Room Service,Room Service,Room Service (24 Hours),STD / ISD,Safari,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Shopping Drop Facility (on fixed timings),Ski Storage,Skiing Facility Available,Smoke Detector,Smoking Area,Smoking Area,Smoking Rooms,Solarium,Spa On Site - Chargeable,Spa Services Nearby,Spa Services Onsite - Free,Speciality Restaurant,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suitable For Children,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Transfers Available,Translation Services,Trekking,Valet service,Veg / Non Veg Kitchens Separate,Vegetarian Food / Jain Food Available,Veranda,Village Visit ( Chargeable ),Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,Yoga Hall / Classes
0,H000001,Baragarh Regency,The standard check-i

In [17]:
# merge dataframe utama+hotel facilities dengan room facilities
df_final = pd.concat([df_final.reset_index(drop=True),df_room_facilities.reset_index(drop=True)], axis=1)
print(df_final.shape)
df_final.head(3)

(1953, 383)


,property_id,property_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,hotel_facilities,room_facilities,site_review_rating,Bike on Rent,Cook Services,Drivers Rest Room,24 Hour Front Desk,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Airport Transfer Available / Surcharge,Airport Transfer Free,Amphitheatre,Anti-Slip Ramps,Audio - Visual Equipment,Ayurvedic Facilities,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Boating,Bowling,Business Center,Business Center,CCTV,Camel Ride,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Conference Facility,Courier Service,Courtesy Coach,Currency Exchange,Dance Performances (on demand),Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Exit Signs,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Food Facility,Free Internet Access,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,Horse Ride ( Chargeable ),Hotel Trading License,In House Events,Internet / Fax (Reception area only),Internet Access,Internet Access,Internet Access - Surcharge,Internet Access Free,Jacuzzi,Karaoke,Karaoke,Kids Play Zone,Kitchen available (home cook food on request),Laptop Available on Hire,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Luggage Storage,Major Credit Cards Accepted,Marina On Site,Massage Services,Massage Services,Medical Assistance Available,Meditation Room,Metal Detectors,Mini Theatre,Multi - Line Phone,Multi Cuisine Restaurant,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Newspapers In Lobby,Night Club,Non Smoking Hotel,Non Smoking Rooms,Open Air Restaurant / Dining,Open Air Restaurant / Dining,Outdoor Parking - Non Secured,Outdoor Parking - Secured,Oxygen Cylinder (on request),Paid Transfers,Parking (Surcharge),Parking Facilities Available,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Cafe,Pool Table,Poolside Bar,Porter / Bellhop,Porter Services Available,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Room Service,Room Service,Room Service (24 Hours),STD / ISD,Safari,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Shopping Drop Facility (on fixed timings),Ski Storage,Skiing Facility Available,Smoke Detector,Smoking Area,Smoking Area,Smoking Rooms,Solarium,Spa On Site - Chargeable,Spa Services Nearby,Spa Services Onsite - Free,Speciality Restaurant,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suitable For Children,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Transfers Available,Translation Services,Trekking,Valet service,Veg / Non Veg Kitchens Separate,Vegetarian Food / Jain Food Available,Veranda,Village Visit ( Chargeable ),Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,Yoga Hall / Classes,Hot / Cold Running Water,120 AC,220 AC,Air Cond

In [18]:
# save data hotel for ML
# df_final.to_csv('DataHotel_ML.csv')